In [44]:
import pandas as pd
import numpy as np

In [65]:
movies = pd.read_csv('csv_files/movies.csv', encoding='utf-8')
ratings = pd.read_csv('csv_files/ratings.csv', encoding='utf-8')
tags = pd.read_csv('csv_files/tags.csv', encoding='utf-8')

# Opération Movie

In [18]:
movies['list'] = pd.Series(movies['genres'].str.split('|'))

In [19]:
movies.head()

,movieId,title,genres,list
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,[Comedy]


In [47]:
genre_list = [ "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "IMAX", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western", "(no genres listed)" ]
genres = pd.DataFrame({'id': np.arange(1, len(genre_list)+1), 'genre': genre_list })
genres.head()

,id,genre
0,1,Action
1,2,Adventure
2,3,Animation
3,4,Children
4,5,Comedy


In [36]:
genres.to_pickle('pickle_files/genres.pkl')

In [ ]:
rows = []

for idx, row in movies.iterrows():
    movie_id = row['movieId']
    genre_list_for_movie = row['list']

    for genre in genre_list_for_movie:
        try:
            rows.append({'movieId': movie_id, 'genreId': genres.index(genre)})
        except ValueError:
            print(f"This genre was not found: {genre}")
movie_genre = pd.DataFrame(rows)

In [ ]:
expanded_movies = movies.explode('list')
joint_df = expanded_movies.merge(right=genres, how='inner', left_on='list', right_on='genre')
movie_genre = joint_df[['movieId', 'id']]


,movieId,id
0,1,2
1,1,3
2,1,4
3,1,5
4,1,9
...,...,...
154165,292753,8
154166,292755,8
154167,292757,1
154168,292757,2


In [53]:
movie_genre.rename(columns={'id':'genreid'}, inplace=True)
# Ensure column names match PostgreSQL (lowercase)
movie_genre.columns = [col.lower() for col in movie_genre.columns]
movie_genre

C:\Users\stgadmin\AppData\Local\Temp\ipykernel_8460\869974903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_genre.rename(columns={'id':'genreid'}, inplace=True)


,movieid,genreid
0,1,2
1,1,3
2,1,4
3,1,5
4,1,9
...,...,...
154165,292753,8
154166,292755,8
154167,292757,1
154168,292757,2


In [54]:
movie_genre.to_pickle('pickle_files/movie_genre.pkl')

In [24]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)\s*$')
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)\s*$', '', regex=True)

movies.head()

,movieId,title,genres,list,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,Comedy|Romance,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,Comedy,[Comedy],1995


In [32]:
movies_df = movies[['movieId', 'title', 'year']]
movies_df

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
87580,292731,The Monroy Affaire,2022
87581,292737,Shelter in Solitude,2023
87582,292753,Orca,2023
87583,292755,The Angry Breed,1968


In [33]:
movies_df.to_pickle('pickle_files/movies.pkl')

# Opération ratings

In [25]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [68]:
# ratings['rating'] = ratings['rating'].apply(lambda x: x*2).astype(int)
ratings.rename({'timestamp':'recorded_at'}, inplace=True)
ratings.columns = [col.lower() for col in ratings.columns]
ratings

,userid,movieid,rating,timestamp
0,1,17,8,944249077
1,1,25,2,944250228
2,1,29,4,943230976
3,1,30,10,944249077
4,1,32,10,943228858
...,...,...,...,...
32000199,200948,79702,9,1294412589
32000200,200948,79796,2,1287216292
32000201,200948,80350,1,1294412671
32000202,200948,80463,7,1350423800


In [67]:
ratings.to_pickle('pickle_files/ratings.pkl')

# Operation tags

In [27]:
tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


In [ ]:
tags.rename({'timestamp':'recorded_at'})
tags.columns = [col.lower() for col in tags.columns]

# User

In [56]:
user_id = ratings['userId'].unique()
user_id

array([     1,      2,      3, ..., 200946, 200947, 200948],
      shape=(200948,))

In [57]:
tags['userId'].unique()

array([    22,     34,     55, ..., 162274, 162278, 162279],
      shape=(15848,))

In [80]:
for idx, row in tags.iterrows():
    if row['userId'] not in user_id:
        print(row['movieId'])

Sans output, tags ne contient pas d'userId exclusif. On peut utiliser que les userId de ratings

In [60]:
app_user = pd.DataFrame([], columns=['userid', 'username', 'pw'])
app_user['userid'] = user_id

In [61]:
app_user.head()

,userid,username,pw
0,1,NaN,NaN
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN


In [63]:
app_user.to_pickle('pickle_files/app_user.pkl')